# Índice
1. [Introdução](#introduction)
2. [Lendo os dados](#reading)   
3. [Visão Geral](#visao)    
 3.1. [Mapas de Calor](#seaborn)
4. [Pré Processamento](#processing)   
 4.1. [Agrupamento dos dados para teste](#grouping)   
 4.1.1. [Criando nova coluna "Season"](#column-season)   
 4.2. [Criando DataFrames separados para cada estação](#dataframe-season)   
5. [SVR (Support Vector Regression)](#svr)
6. [Referências](#references)

## 1. Introdução <a name="introduction"></a>
<p>Este <i>dataset</i> contém dados meteorológicos do Aeroporto Internacional de Raleigh Durham retirados do serviço da Web do NOAA.</p>

## 2. Lendo os Dados <a name="reading"></a>

In [ ]:
import pandas as pd

# As colunas estão separadas pelo símbolo ';' #. 
weather_history = pd.read_csv('rdu-weather-history.csv', parse_dates=['date'], sep=';')

## 3. Visão geral <a name="visao"></a>
<p>O <i>dataset</i> contém informação sobre dados meteorológicos do Aeroporto Internacional de Raleigh Durham, desde 2007. Possui dados como temperaturas mínimas e máximas, volume de chuva, velocidade do vento e etc.</p>   
</p>Visualizando as primeiras cinco entradas do DataFrame:</p>    

In [ ]:
weather_history.head()

In [ ]:
# Número de entradas
print("Número de linhas do dataset:", weather_history.shape[0])

print("\n\nResumo estatístico do DataFrame:")
weather_history.describe()

In [ ]:
weather_history.info()

## 3.1 Utilizando a função Seaborn de mapas de calor <a name="seaborn"></a>
<p>Seaborn é uma biblioteca em python para criação de visualizações estatísticas.</p>
<p>O método ```.corr() ```, calcula o <b>coeficiente de correlação de Pearson</b> entre todos os pares de colunas numéricas do DataFrame.</p>
<p>Interpretando o coeficiente:
- 0.9 para mais ou para menos indica uma correlação muito forte.
- 0.7 a 0.9 positivo ou negativo indica uma correlação forte.
- 0.5 a 0.7 positivo ou negativo indica uma correlação moderada.
- 0.3 a 0.5 positivo ou negativo indica uma correlação fraca.
- 0 a 0.3 positivo ou negativo indica uma correlação desprezível.

In [ ]:
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

plt.figure(figsize=(18,10))
weather_map = sns.heatmap(weather_history.corr(), annot=True, square=True, cmap="YlGnBu", linewidths=.3)


## 4. Pré-processamento dos dados <a name="processing"></a>

In [ ]:
# Substituindo as string por números binários -- No = 0.0 e Yes = 1.0

weather_history = weather_history.replace('No', 0.0)
weather_history = weather_history.replace('Yes', 1.0)

In [ ]:
# Verificando a soma dos valores nulos em cada feature

weather_history.isnull().sum()

In [ ]:
from sklearn.preprocessing import Imputer
import numpy as np

#Substituindo os valores nulos pela média dos valores referentes a feature
imputer = Imputer(missing_values=np.nan, strategy='median', axis=0)
weather_history[['temperaturemin']] = imputer.fit_transform(weather_history[['temperaturemin']])
weather_history[['temperaturemax']] = imputer.fit_transform(weather_history[['temperaturemax']])
weather_history[['precipitation']] = imputer.fit_transform(weather_history[['precipitation']])
weather_history[['snowfall']] = imputer.fit_transform(weather_history[['snowfall']])
weather_history[['snowdepth']] = imputer.fit_transform(weather_history[['snowdepth']])
weather_history[['avgwindspeed']] = imputer.fit_transform(weather_history[['avgwindspeed']])
weather_history[['fastest2minwinddir']] = imputer.fit_transform(weather_history[['fastest2minwinddir']])
weather_history[['fastest2minwindspeed']] = imputer.fit_transform(weather_history[['fastest2minwindspeed']])
weather_history[['fastest5secwinddir']] = imputer.fit_transform(weather_history[['fastest5secwinddir']])
weather_history[['fastest5secwindspeed']] = imputer.fit_transform(weather_history[['fastest5secwindspeed']])

weather_history.isnull().sum()

## 4.1. Agrupamento dos dados para teste    <a name="grouping"></a>

Como o clima de uma região possui uma grande relação com as estações do ano, os dados para teste foram agrupados de acordo com a estação que occorre em uma data específica.

- Primavera: 20 de março a 20 de junho 
- Verão: 21 de junho a 21 de setembro
- Outono: 22 de setembro a 20 de dezembro
- Inverno: 21 de dezembro a 19 de março

### 4.1.1. Criando nova coluna "Season"    <a name="column-season"></a>

A coluna "season" foi criada com base nos periodos de inicio e de fim das estações do ano no local em que os dados foram colhidos.

Antes de utilizar o algoritimo para definir as estações a partir das datas é feita uma pequena alteração no formato da data.  
Antes: AAAA-MM-DD  
Depois: AAAAMMDD

In [ ]:
season_array = []

for date in weather_history['date']:
    if int(date.strftime('%m%d')) >= 1221 or int(date.strftime('%m%d')) <= 319:
        season = 0 #'winter'
    elif int(date.strftime('%m%d')) >= 320 and int(date.strftime('%m%d')) <= 620:
        season = 1 #'spring'
    elif int(date.strftime('%m%d')) >= 621 and int(date.strftime('%m%d')) <= 921:
        season = 2 #'summer'
    elif int(date.strftime('%m%d')) >= 922 and int(date.strftime('%m%d')) <= 1220:
        season = 3 #'autumn'    
    
    season_array.append(season)

weather_history['season'] = season_array

In [ ]:
#Ordenando o dataset pela coluna data

weather_history = weather_history.sort_values('date', ascending=True)
weather_history.head()

## 4.2. Criando DataFrames separados para cada estação. <a name="dataframe-season"></a>

In [ ]:
winter_weather_history = weather_history[weather_history['season'] == 0]
spring_weather_history = weather_history[weather_history['season'] == 1]
summer_weather_history = weather_history[weather_history['season'] == 2]
autumn_weather_history = weather_history[weather_history['season'] == 3]

In [ ]:
len_winter = len(winter_weather_history['date'])
len_summer = len(summer_weather_history['date'])
len_spring = len(spring_weather_history['date'])
len_autumn = len(autumn_weather_history['date'])

X = ['winter', 'summer', 'spring', 'autumn']
Y = [len_winter, len_summer, len_spring, len_autumn]

plt.bar(X, Y)
plt.xlabel('Seasons')
plt.ylabel('Amount of data for each season')
plt.title('Season per amount')
plt.show()

In [ ]:
# Exemplo para mostrar que a correlação com a divisão do dataframe em estações aumentou. Nesse exemplo
# tem os dados do verão de 2007

plt.figure(figsize=(10,8))

summer_weather_history = summer_weather_history[summer_weather_history['date'] <= '2007-12-31']                                        
summer_weather_history = summer_weather_history.drop(columns=['snowfall', 'snowdepth', 'ice', 'glaze', 'freezingrain', 'highwind', 'hail', 'blowingsnow', 'dust', 'freezingfog', 'season', 'snow'])
sns.heatmap(summer_weather_history.corr(), square=True, cmap="YlGnBu", linewidths=.2)

## 5. SVR (Support Vector Regression)  <a name="svr"></a>  

In [ ]:
import matplotlib.pyplot as plt
import sys  

reload(sys)  
sys.setdefaultencoding('utf8')

X = weather_history.drop('temperaturemin', axis=1).values
y = weather_history['temperaturemin'].values

weather_history_no_min = weather_history.drop('temperaturemin', axis=1)

X_new = weather_history_no_min
for i in enumerate(weather_history_no_min.columns):
    related_value = weather_history_no_min[i[1]].values
    if not np.array_equal(np.unique(related_value), [0, 1.]):
        print(i)
        plt.scatter(related_value, y)
        plt.ylabel(str('Temperatura mínima'))
        plt.xlabel(str(i[1]))
        plt.show()
    else:
        X_new = X_new.drop(i[1], axis=1)

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

X_new_new = X_new.drop('date', axis=1)
X_new_new = X_new_new.values
regr = SVR(kernel='linear')
y = y.reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X_new_new, y, test_size=0.25, random_state=42)
regr.fit(X_train, y_train)

In [ ]:
regr.score(X_train, y_train)

In [ ]:
predict = regr.predict(X_test)
coeficient = regr.coef_
figure, axes = plt.subplots(figsize=(9, 9))
ax = sns.regplot(x = y_test, y = predict, scatter_kws={"color": "orange"}, line_kws={"color": "blue"})
ax = ax.set(xlabel='Actual Temperaturemin: Test Set', ylabel = 'Predicted Temperaturemin: Test Set')

## 6. Referências  <a name="references"></a>  

> [Coeficiente de correlação de Pearson](https://pt.wikipedia.org/wiki/Coeficiente_de_correla%C3%A7%C3%A3o_de_Pearson#Refer%C3%AAncias)